In [1]:
import pandas as pd
import numpy as np
import spacy
pd.set_option('display.max_colwidth', None)

In [2]:
# !python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.4 MB 18 kB/s s eta 0:00:01     |████████████████▏               | 393.6 MB 16.2 MB/s eta 0:00:24     |███████████████████████████▊    | 672.8 MB 1.1 MB/s eta 0:01:32
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dragonzord/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
patient_notes = pd.read_csv("patient_notes.csv")
patient_notes.head(2)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. ."


In [17]:
features = pd.read_csv("features.csv")
features.head(2)

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction
1,1,0,Family-history-of-thyroid-disorder


In [18]:
second_note = patient_notes.iloc[1, 2]
second_feature = features.iloc[1, 2]

In [19]:
print(second_note)
print(second_feature)

17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. .
Family-history-of-thyroid-disorder


In [20]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())

In [21]:
import prepare_jag
clean_note = prepare_jag.basic_clean2(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])

In [22]:
len(clean_note)

965269

### Using spacy

In [23]:
nlp = spacy.load("en_core_web_lg")

In [24]:
doc = nlp(clean_note)

In [25]:
len(doc)

153373

In [ ]:
for token in doc[:100]:
    if token.pos_ == "NOUN" or token.pos_ == "VERB":
        print(token.text, token.ent_iob_, token.ent_type_, token.pos_)

In [29]:
for token in doc[:100].noun_chunks:
    print(token)
    print({tok.text: tok.pos_ for tok in token if tok.pos_ not in ["NUM", "PUNCT", "SYM"]})

17-year-old male come student health clinic
{'year': 'NOUN', 'old': 'ADJ', 'male': 'NOUN', 'come': 'VERB', 'student': 'NOUN', 'health': 'NOUN', 'clinic': 'NOUN'}
heart
{'heart': 'NOUN'}
mr cleveland's mother
{'mr': 'PROPN', 'cleveland': 'PROPN', "'s": 'PART', 'mother': 'NOUN'}
verbal consent history physical examination treatment
{'verbal': 'ADJ', 'consent': 'NOUN', 'history': 'NOUN', 'physical': 'ADJ', 'examination': 'NOUN', 'treatment': 'NOUN'}
2-3 months
{'months': 'NOUN'}
-associated dispnea exersion
{'-associated': 'VERB', 'dispnea': 'NOUN', 'exersion': 'NOUN'}
fe
{'fe': 'PROPN'}
heart
{'heart': 'NOUN'}
-pmhnonmeds
{'-pmhnonmeds': 'NOUN'}
friendnkda
{'friendnkda': 'NOUN'}
mi recentlymother thyroid dz -shnon-smokermariguana 5-6 months
{'mi': 'PROPN', 'recentlymother': 'X', 'thyroid': 'PROPN', 'dz': 'NOUN', '-shnon': 'ADV', 'smokermariguana': 'ADJ', 'months': 'NOUN'}
3 - 4 min
{'min': 'NOUN'}


In [ ]:
colors = {"Annotation" :"linear-gradient(to right, #2980b9, #6dd5fa, #ffffff);" } 
options = {"colors": colors}
spacy.displacy.render(doc, style='ent', options=options , manual=True, jupyter=True);

In [ ]:
for ent in doc[:10000].ents:
    print(f"text: {ent.text}, start_char: {ent.start_char}, end_char: {ent.end_char}, label_: {ent.label_}, start: {ent.start}, ent_id: {ent.ent_id_}, label: {ent.label}, vector_norm: {ent.vector_norm}")

### Using Scispacy
#### scispacy is not working in this environment

In [14]:
#%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

In [17]:
# print("works1")
# import scispacy
# print("works2")

# from scispacy.linking import EntityLinker

# print("works3")
# sci_nlp = spacy.load("en_core_sci_lg")
# # This line takes a while, because we have to download ~1GB of data
# # and load a large JSON file (the knowledge base). Be patient!
# # Thankfully it should be faster after the first time you use it, because
# # the downloads are cached.
# print("works4")


In [18]:

# # NOTE: The resolve_abbreviations parameter is optional, and requires that
# # the AbbreviationDetector pipe has already been added to the pipeline. Adding
# # the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# # that linking will only be performed on the long form of abbreviations.
# sci_nlp.add_pipe("scispacy_linker", config={"linker_name": "umls"})

# doc = sci_nlp(clean_note)
# # Let's look at a random entity!
# # doc comes from above
# entity = doc.ents[1]
# print(f"Name: {entity}")

# # Each entity is linked to UMLS with a score
# # (currently just char-3gram matching).
# linker = nlp.get_pipe("scispacy_linker")
# for umls_ent in entity._.kb_ents:
# 	print(linker.kb.cui_to_entity[umls_ent[0]])